# --- Day 17: Chronospatial Computer ---


In [56]:
# --- Part One ---

filename = "input.txt"

#filename = "test.txt" # decomment to test

class Computer:
    
    def __init__(self):
        # integer registers
        self.registers = { "A": 0, "B": 0, "C": 0}
        # program memory
        self.program = []
        # output
        self.outputs = []

    def set_registers(self, a, b=0, c=0):
        self.registers = { "A": a, "B": b, "C": c}

    def load_file(self, file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
        # parse registers
        for line in lines[:3]: #First three lines for registers
            key, value = line.split(':')
            reg_name = key.strip().split(" ")[1]
            self.registers[reg_name] = int(value.strip())
        # parse program
        program_line = lines[4]
        self.program = list(map(int, program_line.split(':')[1].strip().split(',')))

    def run(self, a= None, b=0, c=0):

        if a != None:
            self.set_registers(a,b,c)
        #--- helper to resolve combo operands
        def combo(operand):
            if operand <= 3: return operand
            elif operand == 4: return self.registers['A']
            elif operand == 5: return self.registers['B']
            elif operand == 6: return self.registers['C']
            else: raise ValueError("Invalid combo operand: 7 is reserved")

        ip = 0  # instruction pointer
        self.outputs = []
        # Run the program
        while ip < len(self.program):
            opcode = self.program[ip]
            operand = self.program[ip +1] if ip + 1 < len(self.program) else 0

            if opcode == 0:  # adv: A = A // (2 ** resolve_combo(operand))
                self.registers['A'] //= 2 ** combo(operand)
            elif opcode == 1:  # bxl: B = B ^ operand (literal)
                self.registers['B'] ^= operand
            elif opcode == 2:  # bst: B = resolve_combo(operand) % 8
                self.registers['B'] = combo(operand) % 8
            elif opcode == 3:  # jnz: if A != 0, ip = operand (literal)
                if self.registers['A'] != 0:
                    ip = operand
                    continue
            elif opcode == 4:  # bxc: B = B ^ C
                self.registers['B'] ^= self.registers['C']
            elif opcode == 5:  # out: outputs.append(resolve_combo(operand) % 8)
                self.outputs.append(combo(operand) % 8)
            elif opcode == 6:  # bdv: B = A // (2 ** resolve_combo(operand))
                self.registers['B'] = self.registers['A'] // (2 ** combo(operand))
            elif opcode == 7:  # cdv: C = A // (2 ** resolve_combo(operand))
                self.registers['C'] = self.registers['A'] // (2 ** combo(operand))
            else:
                raise ValueError(f"Invalid opcode: {opcode}")
            # next instruction
            ip +=2

        return self.outputs


# ---- main ------

computer = Computer()

computer.load_file(filename)

res1 = ','.join(map(str, computer.run()))
print(f"The solution for part 1 is: {res1}")


The solution for part 1 is: 1,5,0,3,7,3,0,3,1


In [97]:
# --- Part Two ---
# Uses code op Part One

# analizing the program
#
# Program: 2,4,1,5,7,5,1,6,0,3,4,1,5,5,3,0
#
#  Line | Meaning  
#  2 ,4 | b = a % 8
#  1, 5 | b = b ^ 5
#  7, 5 | c = a >> b
#  1, 6 | b = b ^ 6
#  0, 3 | a = a >> 3
#  4, 1 | b = b ^ c
#  5, 5 | out b (if a!=0)
#  3, 0 | loop

filename = "input.txt"

# filename = "test2.txt" # decomment to test


# ---- main ------

computer = Computer()

computer.load_file(filename)

# Analyzing the program starting from the end and look for all the values of A that result in
# the last digit of the program. A can only be between 0 and 7. In the code, the last 3 bits are then
# removed from A (command (0,3)).
# - The last digit to search for is “0”.
#   The only candidate is [3] because computer.run(3) -> 0.
# - Moving to the next digit: for each candidate, I start from the value candidate * 8 (candidate << 3).
#   Then I search for all candidates that correctly return the last two digits of the program.
#   In our case [3, 0].
#   the candidates are [24, 25, 29, 31]
# - continue untill all the digits are checked
 
program = computer.program
print (program)

to_check = [0]
for l in range(len(program)):
    next_to_check = []
    for a in to_check:
        for i in range(8):
            new_a = (a << 3) + i
            if computer.run(new_a) == program[-1-l:]:
                next_to_check.append(new_a)
    to_check = next_to_check
    print(to_check)

ret2 = min(to_check)

# check if res2 is correct
print(computer.run(ret2))

print(f"The solution for part 2 is: {ret2}")



[2, 4, 1, 5, 7, 5, 1, 6, 0, 3, 4, 1, 5, 5, 3, 0]
[3]
[24, 25, 29, 31]
[192, 196, 198, 199, 200, 249]
[1538, 1540, 1542, 1543, 1570, 1572, 1575, 1586, 1588, 1594, 1602, 1604, 1606, 1607, 1992]
[12306, 12311, 12322, 12326, 12337, 12338, 12343, 12345, 12346, 12348, 12567, 12582, 12601, 12604, 12695, 12710, 12759, 12823, 12838, 12849, 12855, 12857, 12860, 15936, 15939]
[98579, 98611, 98702, 98766, 98784, 98785, 100659, 100814, 100832, 100833, 101683, 102074, 102707, 102798, 102862, 102880, 102881]
[788633, 788637, 788639, 788889, 788893, 788895, 789621, 789622, 790133, 790134, 790272, 790277, 790280, 790285, 805273, 805277, 805279, 806517, 806518, 806656, 806661, 806664, 806669, 813465, 813469, 813471, 816597, 821657, 821661, 821663, 822389, 822390, 822901, 822902, 823040, 823045, 823048, 823053]
[6309066, 6309098, 6309102, 6309114, 6311114, 6311146, 6311150, 6311162, 6316969, 6316974, 6321065, 6321070, 6322179, 6322217, 6322219, 6322222, 6322280, 6322286, 6442186, 6442218, 6442222, 644223

In [91]:
s = 24
print(f"{s+0} -> {computer.run(s+0)}")
print(f"{s+1} -> {computer.run(s+1)}")
print(f"{s+2} -> {computer.run(s+2)}")
print(f"{s+3} -> {computer.run(s+3)}")
print(f"{s+4} -> {computer.run(s+4)}")
print(f"{s+5} -> {computer.run(s+5)}")
print(f"{s+6} -> {computer.run(s+6)}")
print(f"{s+7} -> {computer.run(s+7)}")


24 -> [3, 0]
25 -> [3, 0]
26 -> [1, 0]
27 -> [0, 0]
28 -> [1, 0]
29 -> [3, 0]
30 -> [6, 0]
31 -> [3, 0]
